In [ ]:


import json
import os
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.document import Document
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.runnables import RunnableConfig

OPENAI_API_KEY = "sk-GqA4Uj6iZXaykbOzIlFGtmdJr6VqiX94NhhjPZaf81kylRzh"
OPENAI_API_BASE = "https://api.opentyphoon.ai/v1"
MODEL_NAME = "typhoon-v2-70b-instruct"
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
JSON_PATH = "Jsonfile\M.JSON"
SIMILARITY_THRESHOLD = 0.7

def format_value(value):
    if isinstance(value, list):
        return "\n".join([f"- {item}" for item in value]) if value else "ไม่มีข้อมูล"
    elif isinstance(value, dict):
        return "\n".join([f"  {k}: {format_value(v)}" for k, v in value.items()]) if value else "ไม่มีข้อมูล"
    elif value is None:
        return "ไม่มีข้อมูล"
    else:
        return str(value).replace("\\n", "\n")

def parse_json_to_docs(data, parent_key="", docs=None):
    if docs is None:
        docs = []
    if isinstance(data, dict):
        current_topic = data.get("หัวข้อ", data.get("หัวข้อย่อย", parent_key))
        content_parts = []
        metadata = {"source": parent_key.strip('.')}

        for key, value in data.items():
            current_key = f"{parent_key}{key}" if parent_key else key
            if isinstance(value, (dict, list)) and key not in ["หัวข้อ", "หัวข้อย่อย"]:
                parse_json_to_docs(value, f"{current_key}.", docs)
            elif key not in ["หัวข้อ", "หัวข้อย่อย"]:
                readable_key = key.replace("_", " ").replace("เป้า ", "เป้าหมาย ")
                content_parts.append(f"{readable_key}: {format_value(value)}")

        if content_parts:
            page_content = f"หัวข้อ: {current_topic}\n" + "\n".join(content_parts)
            docs.append(Document(page_content=page_content.strip(), metadata=metadata))
    elif isinstance(data, list) and parent_key:
        page_content = f"หัวข้อ: {parent_key.strip('.')}\n{format_value(data)}"
        metadata = {"source": parent_key.strip('.')}
        docs.append(Document(page_content=page_content.strip(), metadata=metadata))
    return docs

def extract_questions_from_docs(docs):
    questions = []
    for doc in docs:
        lines = doc.page_content.split("\n")
        for line in lines:
            if "?" in line and len(line.strip()) > 10:
                questions.append(line.strip())
    return list(set(questions))

with open(JSON_PATH, "r", encoding="utf-8") as f:
    policy_data = json.load(f)

documents = parse_json_to_docs(policy_data["นโยบายสินเชื่อ_รวม"])
model_kwargs = {'device': 'cpu'}
encode_kwargs = {}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="Represent this query for retrieving relevant documents: "
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE,
    model_name=MODEL_NAME,
    temperature=0.5,
    max_tokens=8192,
)

vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

prompt_template = """
คุณคือผู้ช่วย AI ที่เชี่ยวชาญด้านนโยบายสินเชื่อ กรุณาตอบคำถามต่อไปนี้โดยใช้ข้อมูลที่ให้มาเท่านั้น:

ข้อมูลที่เกี่ยวข้อง (Context):
{context}

คำถาม:
{input}

คำตอบ (เป็นภาษาไทย):
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
candidate_questions = extract_questions_from_docs(documents)
qa_results = []

for question in candidate_questions:
    retrieved_docs = retriever.get_relevant_documents(question)
    if len(retrieved_docs) == 0:
        continue
    response = retrieval_chain.invoke({"input": question}, config=RunnableConfig(tags=["batch_run"]))
    if "answer" in response and response["answer"].strip():
        seen_sources = set()
        for doc in response.get("context", []):
            source = doc.metadata.get("source", "ไม่ทราบแหล่งที่มา")
            if source not in seen_sources:
                print(f"- {source}")
                seen_sources.add(source)

        qa_results.append({
            "question": question,
            "answer": response["answer"],
            "sources": list(seen_sources)
        })

with open("qa_results.json", "w", encoding="utf-8") as f:
    json.dump(qa_results, f, ensure_ascii=False, indent=2)

while True:
    user_query = input("คำถามของคุณ: ")
    if user_query.lower() in ["exit", "quit", "ออก"]:
        print("กำลังออกจาก Chatbot...")
        break
    if not user_query.strip():
        continue

    print("กำลังค้นหาคำตอบ...")
    response = retrieval_chain.invoke({"input": user_query})

    print("\nคำตอบ:")
    print(response["answer"])

    print("\nแหล่งข้อมูลที่เกี่ยวข้อง:")
    seen_sources = set()
    for doc in response.get("context", []):
        source = doc.metadata.get("source", "ไม่ทราบแหล่งที่มา")
        if source not in seen_sources:
            print(f"- {source}")
            seen_sources.add(source)
    print("--------------------------------------------------")


In [ ]:
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import faiss
import os

# ---- Configs ---- #
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# ---- File Path ---- #
FILE_PATH = "Data real/Car rate book.xlsx"  # ✅ Hardcoded Excel path

# ---- Load Excel & Chunk to Docs ---- #
def load_docs(file_path):
    loader = UnstructuredExcelLoader(file_path)
    return loader.load()

# ---- Load Embeddings ---- #
def load_embeddings():
    return HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True}
    )

# ---- Build Vector DB ---- #
def build_vector_store(docs, embeddings):
    index = faiss.IndexFlatL2(len(embeddings.embed_query(" ")))
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={}
    )
    vector_store.add_documents(docs)
    return vector_store

# ---- Load LLM ---- #
def load_llm():
    return ChatOpenAI(
        openai_api_key="sk-GqA4Uj6iZXaykbOzIlFGtmdJr6VqiX94NhhjPZaf81kylRzh",  # Replace with your actual key
        openai_api_base="https://api.opentyphoon.ai/v1",
        model_name="typhoon-v2-70b-instruct",
        temperature=0.5,
        max_tokens=1024
    )

# ---- Setup RAG Chain ---- #
def build_rag_chain(llm, retriever):
    system_prompt = (
        "You are a helpful assistant for answering questions based on Excel data.\n"
        "Use the context to answer the question.\n"
        "If unsure, say 'I don't know'. Keep it concise.\n\n{context}"
    )
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}")
    ])
    qa_chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(retriever, qa_chain)

# ---- CLI ---- #
def main():
    print(f"📄 Loading and processing: {FILE_PATH}")
    docs = load_docs(FILE_PATH)
    embeddings = load_embeddings()
    vector_store = build_vector_store(docs, embeddings)
    retriever = vector_store.as_retriever()
    llm = load_llm()
    rag_chain = build_rag_chain(llm, retriever)

    print("\n✅ Ready! Ask questions about the Excel data.\n")
    while True:
        question = input("❓ Your question: ").strip()
        if not question:
            break
        result = rag_chain.invoke({"input": question})
        print("🧠 Answer:", result.get("answer", "No answer found"))
        print("-" * 50)

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd
import json
import os

excel_file_path = "Data real/Car rate book.xlsx"
output_dir = os.path.dirname(excel_file_path) or '.'
json_output_path = os.path.join(output_dir, "car_rate_book_output.json")

try:
    df = pd.read_excel(excel_file_path, engine='openpyxl')
    json_data_string = df.to_json(
        orient='records',
        indent=4,
        force_ascii=False,
        date_format='iso',
        default_handler=str
    )
    os.makedirs(output_dir, exist_ok=True)
    with open(json_output_path, 'w', encoding='utf-8') as f:
        f.write(json_data_string)
except Exception:
    pass


In [ ]:
import pandas as pd
import io # สำหรับอ่านข้อมูล string เป็นไฟล์
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
import faiss # แม้ FAISS จะถูกเรียกผ่าน LangChain แต่ import ไว้เผื่อกรณี debug ขั้นสูง

# ---- Configs ---- #
os.environ["TOKENIZERS_PARALLELISM"] = "false" # ตั้งค่าตามที่ระบุในโค้ดเดิม

# ---- ข้อมูล CSV (จำลองจาก Excel text ที่ให้มา) ---- #
csv_data = """DB,FDATEA,LDATEA,MODELCOD,MODELDESC,MANUYR,LEASETYPE,TARGETSTAT,RATE,USERATE,FEE504,FEE508,FEE512,FEE514,FEE515,BLOCK,TYPECOD,GCODE,RATEESTIMATE,RATEMARKET,USERCREATE,GEAR,PRODUCT GROUP
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2019,,,750000,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2018,,,612300,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2017,,,410800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2016,,,410800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2015,,,440700,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2014,,,400400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2013,,,371800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2012,,,356200,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2011,,,287300,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2010,,,284700,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2009,,,267800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2008,,,240500,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2007,,,239200,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2006,,,240500,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.7 เกียร์ออโต้,2005,,,230100,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2015,,,400400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2014,,,462800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2013,,,423800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2012,,,408200,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2011,,,344500,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2010,,,328900,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2009,,,313300,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2008,,,288600,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2007,,,283400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2006,,,270400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ออโต้,2005,,,267800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ธรรมดา,2010,,,317200,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ธรรมดา,2009,,,293800,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ธรรมดา,2008,,,260000,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ธรรมดา,2007,,,256100,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ธรรมดา,2006,,,243100,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,3.0 เกียร์ธรรมดา,2005,,,231400,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.8 เกียร์ออโต้,2023,,,971100,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.8 เกียร์ออโต้,2022,,,926900,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.8 เกียร์ออโต้,2021,,,855400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.8 เกียร์ออโต้,2020,,,709800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.8 เกียร์ออโต้,2019,,,620100,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.8 เกียร์ออโต้,2018,,,422500,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.8 เกียร์ออโต้,2017,,,410800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.8 เกียร์ออโต้,2016,,,410800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.8 เกียร์ออโต้,2015,,,400400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ออโต้,2015,,,471900,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ออโต้,2014,,,435500,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ออโต้,2013,,,429000,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ออโต้,2012,,,400400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ธรรมดา,2015,,,414700,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ธรรมดา,2014,,,383500,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ธรรมดา,2013,,,377000,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ธรรมดา,2012,,,348400,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ธรรมดา,2011,,,293800,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ธรรมดา,2010,,,282100,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.5 เกียร์ธรรมดา,2009,,,279500,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ออโต้,2023,,,743600,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ออโต้,2022,,,699400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ออโต้,2021,,,656500,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ออโต้,2020,,,601900,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ออโต้,2019,,,432900,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ออโต้,2018,,,422500,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ออโต้,2017,,,410800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ออโต้,2016,,,410800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ออโต้,2015,,,400400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ธรรมดา,2019,,,432900,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ธรรมดา,2018,,,422500,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ธรรมดา,2017,,,410800,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ธรรมดา,2016,,,410800,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,FORTUNER,2.4 เกียร์ธรรมดา,2015,,,486200,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ออโต้,2013,,,172900,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ออโต้,2012,,,161200,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ออโต้,2011,,,153400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ออโต้,2010,,,149500,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ออโต้,2009,,,140400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ออโต้,2008,,,132600,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ออโต้,2007,,,130000,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ออโต้,2006,,,113100,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ธรรมดา,2013,,,167700,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ธรรมดา,2012,,,148200,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ธรรมดา,2011,,,143000,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ธรรมดา,2010,,,137800,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ธรรมดา,2009,,,130000,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ธรรมดา,2008,,,124800,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ธรรมดา,2007,,,124800,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.5 เกียร์ธรรมดา,2006,,,110500,,,,,,,N,TOYOTA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS ATIV,1.2 เกียร์ออโต้,2023,,,280800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS ATIV,1.2 เกียร์ออโต้,2022,,,280800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS ATIV,1.2 เกียร์ออโต้,2021,,,325000,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS ATIV,1.2 เกียร์ออโต้,2020,,,293800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS ATIV,1.2 เกียร์ออโต้,2019,,,239200,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS ATIV,1.2 เกียร์ออโต้,2018,,,226200,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS ATIV,1.2 เกียร์ออโต้,2017,,,215800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2023,,,280800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2022,,,280800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2021,,,336700,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2020,,,317200,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2019,,,257400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2018,,,241800,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2017,,,218400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2016,,,200200,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2015,,,192400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2014,,,180700,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,YARIS,1.2 เกียร์ออโต้,2013,,,166400,,,,,,,N,TOYOTA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2023,,,549900,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2022,,,534300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2016,,,280800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2015,,,270400,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2014,,,260000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2013,,,248300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2012,,,237900,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2011,,,227500,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2010,,,183300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2009,,,176800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2008,,,156000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2007,,,136500,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2006,,,109200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,2.0 เกียร์ออโต้,2005,,,87100,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2023,,,884000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2022,,,828100,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2021,,,781300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2020,,,735800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2019,,,488800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2018,,,427700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2017,,,338000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2016,,,325000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2015,,,304200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2014,,,299000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2013,,,286000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2012,,,273000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2011,,,260000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2010,,,174200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2009,,,159900,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2008,,,156000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2007,,,115700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2006,,,94900,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.0 เกียร์ออโต้,2005,,,85800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2023,,,325000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2022,,,325000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2021,,,325000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2020,,,313300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2019,,,302900,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2018,,,292500,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2017,,,280800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2016,,,265200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2015,,,252200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2014,,,221000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2013,,,197600,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2012,,,182000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2011,,,166400,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2010,,,117000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2009,,,105300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2008,,,78000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2007,,,66300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2006,,,61100,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ออโต้,2005,,,46800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.5 เกียร์ออโต้,2023,,,512200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.5 เกียร์ออโต้,2022,,,499200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.5 เกียร์ออโต้,2021,,,486200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.5 เกียร์ออโต้,2020,,,526500,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.5 เกียร์ออโต้,2019,,,479700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.5 เกียร์ออโต้,2018,,,451100,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.5 เกียร์ออโต้,2017,,,404300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.5 เกียร์ออโต้,2016,,,395200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.5 เกียร์ออโต้,2014,,,217100,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.5 เกียร์ออโต้,2013,,,204100,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,1.5 เกียร์ออโต้,2023,,,731900,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,1.5 เกียร์ออโต้,2022,,,687700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,1.5 เกียร์ออโต้,2021,,,640900,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,1.5 เกียร์ออโต้,2020,,,611000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,1.5 เกียร์ออโต้,2019,,,544700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2021,,,377000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2020,,,351000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2019,,,338000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2018,,,325000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2017,,,312000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2016,,,280800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2015,,,270400,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2014,,,260000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2013,,,247000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2012,,,195000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2011,,,182000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2010,,,145600,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2009,,,135200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2008,,,115700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2007,,,109200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2006,,,92300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ออโต้,2005,,,85800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2016,,,280800,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2015,,,260000,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2014,,,247000,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2013,,,234000,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2012,,,182000,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2011,,,175500,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2010,,,135200,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2009,,,124800,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2008,,,107900,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2007,,,98800,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2006,,,84500,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.8 เกียร์ธรรมดา,2005,,,81900,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.7 เกียร์ออโต้,2005,,,68900,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CIVIC,1.7 เกียร์ธรรมดา,2005,,,67600,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.0 เกียร์ออโต้,2023,,,325000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.0 เกียร์ออโต้,2022,,,313300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.0 เกียร์ออโต้,2021,,,302900,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.0 เกียร์ออโต้,2020,,,292500,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.0 เกียร์ออโต้,2019,,,280800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,3.5 เกียร์ออโต้,2013,,,390000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,3.5 เกียร์ออโต้,2012,,,388700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,3.5 เกียร์ออโต้,2011,,,339300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,3.5 เกียร์ออโต้,2010,,,309400,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,3.5 เกียร์ออโต้,2009,,,278200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,3.5 เกียร์ออโต้,2008,,,258700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,3.5 เกียร์ออโต้,2007,,,244400,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,3.0 เกียร์ออโต้,2007,,,144300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,3.0 เกียร์ออโต้,2006,,,115700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,3.0 เกียร์ออโต้,2005,,,102700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2019,,,494000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2018,,,468000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2017,,,455000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2016,,,313300,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2015,,,302900,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2014,,,292500,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2013,,,280800,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2012,,,270400,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2011,,,239200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2010,,,187200,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2009,,,175500,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2008,,,162500,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2007,,,127400,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2006,,,104000,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,ACCORD,2.4 เกียร์ออโต้,2005,,,89700,,,,,,,N,HONDA,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2019,,,282100,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2018,,,267800,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2017,,,253500,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2016,,,239200,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2015,,,224900,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2014,,,206700,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2013,,,178100,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2012,,,169000,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2011,,,153400,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2010,,,106600,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2009,,,97500,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2008,,,70200,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2007,,,58500,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2006,,,53300,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,CITY,1.5 เกียร์ธรรมดา,2005,,,41600,,,,,,,N,HONDA,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ออโต้,2013,,,299000,,,,,,,N,ISUZU,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ออโต้,2012,,,286000,,,,,,,N,ISUZU,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ออโต้,2011,,,273000,,,,,,,N,ISUZU,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ออโต้,2010,,,254800,,,,,,,N,ISUZU,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ออโต้,2009,,,223600,,,,,,,N,ISUZU,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ออโต้,2008,,,221000,,,,,,,N,ISUZU,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ออโต้,2007,,,195000,,,,,,,N,ISUZU,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ออโต้,2006,,,185900,,,,,,,N,ISUZU,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ออโต้,2005,,,175500,,,,,,,N,ISUZU,CA,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ธรรมดา,2013,,,286000,,,,,,,N,ISUZU,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ธรรมดา,2012,,,274300,,,,,,,N,ISUZU,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ธรรมดา,2011,,,265200,,,,,,,N,ISUZU,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ธรรมดา,2010,,,244400,,,,,,,N,ISUZU,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ธรรมดา,2009,,,213200,,,,,,,N,ISUZU,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ธรรมดา,2008,,,189800,,,,,,,N,ISUZU,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ธรรมดา,2007,,,182000,,,,,,,N,ISUZU,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ธรรมดา,2006,,,169000,,,,,,,N,ISUZU,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,MU7,3.0 เกียร์ธรรมดา,2005,,,163800,,,,,,,N,ISUZU,CA,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2023,,,478400,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2022,,,457600,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2021,,,421200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2020,,,412100,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2019,,,375700,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2018,,,336700,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2017,,,319800,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2016,,,293800,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2015,,,292500,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2023,,,427700,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2022,,,391300,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2021,,,373100,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2020,,,356200,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2019,,,310700,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2018,,,275600,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2017,,,270400,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2016,,,269100,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2015,,,250900,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2023,,,357500,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2022,,,357500,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2021,,,352300,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2020,,,315900,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2019,,,280800,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2018,,,245700,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2017,,,237900,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2016,,,231400,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา,2015,,,213200,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2023,,,357500,,,,,,,N,TOYOTA,P1,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2022,,,357500,,,,,,,N,TOYOTA,P1,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2021,,,357500,,,,,,,N,TOYOTA,P1,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2020,,,357500,,,,,,,N,TOYOTA,P1,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2019,,,323700,,,,,,,N,TOYOTA,P1,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2018,,,312000,,,,,,,N,TOYOTA,P1,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2023,,,432900,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2022,,,432900,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2021,,,432900,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2020,,,443300,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2019,,,382200,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2018,,,374400,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2017,,,339300,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2016,,,330200,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2015,,,325000,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2023,,,432900,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2022,,,418600,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2021,,,399100,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2020,,,387400,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2019,,,352300,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2018,,,348400,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2023,,,518700,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2022,,,475800,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2021,,,448500,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2020,,,417300,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2019,,,447200,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2018,,,417300,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2017,,,369200,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2016,,,334100,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2015,,,325000,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2023,,,432900,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2022,,,432900,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2021,,,430300,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2020,,,409500,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2019,,,356200,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2018,,,336700,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2017,,,315900,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2016,,,297700,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2015,,,296400,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2023,,,540800,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2022,,,540800,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2021,,,525200,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2020,,,517400,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2019,,,470600,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2018,,,456300,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2017,,,396500,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2016,,,373100,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้ ยกสูง,2015,,,365300,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2023,,,501800,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2022,,,490100,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2021,,,477100,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2020,,,440700,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2019,,,406900,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ออโต้,2018,,,365300,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2023,,,581100,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2022,,,525200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2021,,,495300,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2020,,,474500,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2019,,,499200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2018,,,477100,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2017,,,417300,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2016,,,366600,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2015,,,354900,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2023,,,540800,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2022,,,551200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2021,,,492700,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2020,,,462800,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2019,,,419900,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2018,,,390000,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2017,,,352300,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2016,,,330200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX REVO,2.4 เกียร์ธรรมดา ยกสูง,2015,,,317200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ออโต้ ยกสูง,2015,,,322400,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ออโต้ ยกสูง,2014,,,297700,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ออโต้ ยกสูง,2013,,,280800,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ออโต้ ยกสูง,2012,,,274300,,,,,,,N,TOYOTA,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2015,,,326300,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2014,,,266500,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2013,,,270400,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2012,,,257400,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2011,,,237900,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2010,,,234000,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2009,,,222300,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2008,,,205400,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2007,,,195000,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2006,,,188500,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2005,,,185900,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2015,,,291200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2014,,,282100,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2013,,,257400,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2012,,,241800,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2011,,,227500,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2010,,,214500,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2009,,,210600,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2015,,,261300,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2014,,,249600,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2013,,,226200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2012,,,209300,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2011,,,187200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2010,,,182000,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2009,,,171600,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2008,,,161200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2007,,,150800,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2006,,,145600,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2005,,,135200,,,,,,,N,TOYOTA,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ออโต้ ยกสูง,2015,,,291200,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ออโต้ ยกสูง,2014,,,278200,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ออโต้ ยกสูง,2013,,,265200,,,,,,,N,TOYOTA,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2015,,,292500,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2014,,,283400,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2013,,,254800,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2012,,,244400,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2011,,,221000,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2010,,,209300,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2009,,,204100,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2008,,,184600,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2007,,,178100,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2006,,,163800,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2005,,,162500,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2015,,,253500,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2014,,,249600,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2013,,,227500,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2012,,,215800,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2011,,,187200,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2010,,,185900,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา ยกสูง,2009,,,192400,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2015,,,221000,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2014,,,204100,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2013,,,187200,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2012,,,176800,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2011,,,169000,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2010,,,161200,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2009,,,145600,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2008,,,136500,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2007,,,127400,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2006,,,120900,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2005,,,119600,,,,,,,N,TOYOTA,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2015,,,188500,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2014,,,182000,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2013,,,153400,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2012,,,150800,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2011,,,136500,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2010,,,132600,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2009,,,123500,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2008,,,114400,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2007,,,104000,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2006,,,97500,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,HILUX VIGO,2.5 เกียร์ธรรมดา,2005,,,93600,,,,,,,N,TOYOTA,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2015,,,215800,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2014,,,209300,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2013,,,197600,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2015,,,179400,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2014,,,166400,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2013,,,159900,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2012,,,133900,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2011,,,114400,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2010,,,104000,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2009,,,93600,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2008,,,84500,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2007,,,76700,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2006,,,65000,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2005,,,58500,,,,,,,N,ISUZU,P1,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2015,,,274300,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2014,,,271700,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2013,,,245700,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2012,,,213200,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2011,,,187200,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2010,,,176800,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2009,,,166400,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2008,,,156000,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2007,,,140400,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2006,,,132600,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2015,,,248300,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2014,,,244400,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2013,,,224900,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2012,,,191100,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2011,,,158600,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2010,,,156000,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2009,,,146900,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2008,,,132600,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2007,,,118300,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2006,,,115700,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2015,,,214500,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2014,,,208000,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2013,,,188500,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2012,,,162500,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2011,,,139100,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2010,,,132600,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2009,,,114400,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2008,,,106600,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2007,,,97500,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2006,,,88400,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2005,,,83200,,,,,,,N,ISUZU,P2,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ยกสูง,2015,,,270400,,,,,,,N,ISUZU,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ยกสูง,2014,,,258700,,,,,,,N,ISUZU,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ยกสูง,2013,,,253500,,,,,,,N,ISUZU,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ยกสูง,2012,,,223600,,,,,,,N,ISUZU,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ยกสูง,2011,,,215800,,,,,,,N,ISUZU,P2,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2011,,,218400,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2015,,,305500,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2014,,,284700,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2013,,,250900,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2012,,,215800,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2011,,,195000,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2010,,,185900,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2009,,,172900,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2008,,,152100,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2007,,,144300,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ยกสูง,2006,,,137800,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2015,,,248300,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2014,,,235300,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2013,,,211900,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2012,,,183300,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2011,,,170300,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2010,,,153400,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2009,,,146900,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2008,,,130000,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2007,,,117000,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2006,,,100100,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา,2005,,,92300,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ระบบขับเคลื่อน 4 ล้อ,2015,,,340600,,,,,,,N,ISUZU,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ระบบขับเคลื่อน 4 ล้อ,2014,,,317200,,,,,,,N,ISUZU,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ระบบขับเคลื่อน 4 ล้อ,2013,,,302900,,,,,,,N,ISUZU,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ยกสูง,2015,,,309400,,,,,,,N,ISUZU,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ยกสูง,2014,,,300300,,,,,,,N,ISUZU,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ยกสูง,2013,,,278200,,,,,,,N,ISUZU,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ยกสูง,2012,,,247000,,,,,,,N,ISUZU,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ออโต้ ยกสูง,2011,,,232700,,,,,,,N,ISUZU,P4,,,ADMIN,Auto,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2015,,,293800,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2014,,,280800,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2013,,,266500,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
VLOAN,15-ก.ค.-67,15-เม.ย.-68,DMAX,2.5 เกียร์ธรรมดา ระบบขับเคลื่อน 4 ล้อ,2012,,,232700,,,,,,,N,ISUZU,P4,,,ADMIN,Manual,C
"""

# ---- พาธสำหรับบันทึก Vector Store ---- #
FAISS_INDEX_PATH = "faiss_car_rate_index" # ชื่อโฟลเดอร์ที่จะใช้บันทึก index

# ---- ฟังก์ชันสร้าง Documents จาก DataFrame ---- #
def create_documents_from_df(df):
    """
    แปลง DataFrame เป็น List ของ LangChain Document objects.
    แต่ละแถวจะกลายเป็น Document โดยมี page_content เป็น string ของข้อมูลในแถว
    และ metadata เป็น dictionary ของข้อมูลเดิมในแถวนั้นๆ
    """
    documents = []
    for index, row in df.iterrows():
        # สร้าง page_content: รวมข้อมูลสำคัญเป็นข้อความต่อเนื่อง
        # เราอาจเลือกเฉพาะคอลัมน์ที่สำคัญ หรือรวมทั้งหมด
        # การรวมข้อมูลหลายๆ field ช่วยให้ RAG มี context มากขึ้น
        content_parts = []
        for col, value in row.items():
            # แปลงค่าที่ไม่ใช่ null/NaN เป็น string และเพิ่มเข้าไป
            if pd.notna(value):
                 # จัดรูปแบบให้อ่านง่ายขึ้น อาจจะใส่ชื่อคอลัมน์ไปด้วย
                content_parts.append(f"{col}: {str(value)}")
        page_content = ", ".join(content_parts)

        # สร้าง metadata: เก็บข้อมูลเดิมทั้งหมดในรูปแบบ dictionary
        # แปลงค่า NaN เป็น None เพื่อให้เข้ากันได้กับ JSON/Python ทั่วไป
        metadata = row.where(pd.notna(row), None).to_dict()
        metadata['row_index'] = index # เก็บ index เดิมไว้เผื่ออ้างอิง

        documents.append(Document(page_content=page_content, metadata=metadata))
    return documents

# ---- ฟังก์ชันหลัก ---- #
def main():
    # ---- 1. Load Data ---- #
    print("⏳ กำลังอ่านข้อมูลจาก CSV text...")
    try:
        # ใช้ StringIO เพื่ออ่าน string เหมือนเป็นไฟล์ CSV
        # ตั้งค่า dtype=str เพื่ออ่านทุกคอลัมน์เป็นข้อความ ป้องกันปัญหาการแปลงชนิดข้อมูลอัตโนมัติ
        # โดยเฉพาะคอลัมน์วันที่ ที่มีรูปแบบภาษาไทย Pandas อาจจะแปลงผิดพลาดถ้าไม่ระบุ format ชัดเจน
        # การอ่านเป็น string ทั้งหมดก่อนจะปลอดภัยกว่าสำหรับการสร้าง text embedding
        string_io_data = io.StringIO(csv_data)
        df = pd.read_csv(string_io_data, dtype=str)

        # แทนที่ค่าว่างที่ pandas อาจอ่านเป็น NaN ด้วยสตริงว่าง '' หรือ None ตามความเหมาะสม
        # ใช้ fillna('') เพื่อให้แน่ใจว่าทุกค่าเป็น string ก่อนนำไปสร้าง page_content
        df = df.fillna('')

        print(f"✅ อ่านข้อมูลสำเร็จ จำนวน {len(df)} แถว")

    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดระหว่างการอ่านข้อมูล: {e}")
        return # ออกจากฟังก์ชันถ้าอ่านข้อมูลไม่ได้

    # ---- 2. Create Documents ---- #
    print("⏳ กำลังสร้าง LangChain Documents...")
    documents = create_documents_from_df(df)
    print(f"✅ สร้าง Documents สำเร็จ จำนวน {len(documents)} ชุด")
    # แสดงตัวอย่าง Document แรก (เอา comment ออกถ้าต้องการดู)
    # if documents:
    #     print("\nตัวอย่าง Document แรก:")
    #     print("Page Content:", documents[0].page_content)
    #     print("Metadata:", documents[0].metadata)
    #     print("-" * 30)

    # ---- 3. Load Embedding Model ---- #
    print("⏳ กำลังโหลด Embedding model (BAAI/bge-m3)...")
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name="BAAI/bge-m3",
            model_kwargs={"device": "cpu"}, # ใช้ CPU หรือ "cuda" หากมี GPU และต้องการใช้
            encode_kwargs={"normalize_embeddings": True} # แนะนำสำหรับ BGE models
        )
        # ทดสอบ embed คำง่ายๆ เพื่อให้แน่ใจว่าโมเดลโหลดได้
        _ = embeddings.embed_query("test")
        print("✅ โหลด Embedding model สำเร็จ")
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดระหว่างการโหลด Embedding model: {e}")
        return # ออกจากฟังก์ชันถ้าโหลดโมเดลไม่ได้

    # ---- 4. Create FAISS Vector Store ---- #
    print(f"⏳ กำลังสร้าง FAISS index และทำการ Embedding ข้อมูล ({len(documents)} เอกสาร)...")
    # ขั้นตอนนี้จะใช้เวลาสักครู่ ขึ้นอยู่กับจำนวนข้อมูลและพลังประมวลผลของเครื่อง
    try:
        vector_store = FAISS.from_documents(documents, embeddings)
        print("✅ สร้าง FAISS index สำเร็จ")
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดระหว่างการสร้าง FAISS index: {e}")
        return

    # ---- 5. Save FAISS Index ---- #
    print(f"💾 กำลังบันทึก FAISS index ลงในโฟลเดอร์ '{FAISS_INDEX_PATH}'...")
    try:
        vector_store.save_local(FAISS_INDEX_PATH)
        print(f"✅ บันทึก FAISS index สำเร็จที่ '{FAISS_INDEX_PATH}'")
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดระหว่างการบันทึก FAISS index: {e}")

    print("\n🎉 กระบวนการ Embedding และสร้าง Index เสร็จสมบูรณ์!")

# ---- ส่วนเริ่มต้นการทำงานของสคริปต์ ---- #
if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd 
import numpy as np
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings

OPENAI_API_KEY = "sk-GqA4Uj6iZXaykbOzIlFGtmdJr6VqiX94NhhjPZaf81kylRzh"
EXCEL_FILE_PATH = r'Data real\Car rate book.xlsx'
EMBEDDING_MODEL_NAME = 'BAAI/bge-m3'
LLM_MODEL_NAME = "typhoon-v2-70b-instruct"
OPENAI_API_BASE = "https://api.opentyphoon.ai/v1"
VECTOR_STORE_PATH = "car_rate_vectorstore"

CONTNO_TYPE_MAPPING = {
    'T': {'BS': 'BS_รถบัส', 'FT': 'FT_รถไถ', 'HV': 'HV_เครื่องจักรกลหนัก', 
          'OT': 'OT_รถอื่น ๆ', 'T10': 'T10_รถบรรทุก (10 ล้อ)', 
          'T12': 'T12_รถบรรทุก[12ล้อ]', 'T6': 'T6_รถบรรทุก (6 ล้อ)'},
    'A': {'N01': 'N01_สัญญาหลัก-รอง'},
    'C': {'CA': 'CA_รถเก๋ง (2-5 ประตู)', 'P1': 'P1_รถกระบะ (ตอนเดียว)', 
          'P2': 'P2_รถกระบะ (แคป)', 'P4': 'P4_รถกระบะ (4 ประตู)', 
          'T4': 'T4_รถบรรทุก (4 ล้อ)', 'VA': 'VA_รถตู้'},
    'G': {'G01': 'G01_รถไถโรตารี่', 'G03': 'G03_เครื่องยนต์เพื่อการเกษตร'},
    'H': {'LA': 'LA_ที่ดินเปล่า', 'LH': 'LH_ที่ดินพร้อมสิ่งปลูกสร้าง'},
    'I': {'IS': 'IS_Insurance'},
    'L': {'LA': 'LA_ที่ดินเปล่า', 'LH': 'LH_ที่ดินพร้อมสิ่งปลูกสร้าง'},
    'M': {'MC': 'MC_รถมอเตอร์ไซค์'},
    'P': {'P04': 'P04_PLoan_สินเชื่อส่วนบุคคล (กลุ่มบริษัท)'},
    'V': {'HR': 'HR_รถเกี่ยวข้าว'}
}

PRODUCT_GROUP_MAPPING = {
    'A': 'NanoFinance',
    'P': 'PLOAN',
    'T': 'Truck รถบรรทุก',
    'M': 'Motocycle มอเตอร์ไซต์',
    'V': 'รถเกี่ยวข้าว',
    'G': 'โคบูต้า รถไถเดินตาม',
    'H': 'House บ้าน',
    'L': 'Land ที่ดิน',
    'I': 'ประกัน',
    'C': 'รถยนต์'
}

def load_data(file_path):
    df = pd.read_excel(file_path, header=0, dtype=str).fillna('')
    df['MANUYR'] = pd.to_numeric(df['MANUYR'], errors='coerce').astype('Int64')
    df['RATE'] = pd.to_numeric(df['RATE'], errors='coerce').astype('Int64')
    
    try:
        df['FDATEA'] = pd.to_datetime(df['FDATEA'], format='%d-%b-%y', errors='coerce')
        df['LDATEA'] = pd.to_datetime(df['LDATEA'], format='%d-%b-%y', errors='coerce')
    except:
        pass
    
    return df

def format_row(row):
    columns = ['TYPECOD', 'MODELCOD', 'MODELDESC', 'MANUYR', 'GEAR', 'GCODE', 'PRODUCT GROUP', 'RATE']
    labels = {
        'TYPECOD': 'ยี่ห้อ',
        'MODELCOD': 'รุ่นหลัก',
        'MODELDESC': 'รุ่นย่อย',
        'MANUYR': 'ปีที่ผลิต',
        'GEAR': 'เกียร์',
        'GCODE': 'ประเภทรถ',
        'PRODUCT GROUP': 'กลุ่มผลิตภัณฑ์',
        'RATE': 'ราคาประเมิน'
    }

    parts = []
    for col in columns:
        value = row.get(col)
        if pd.notna(value) and value != '':
            label = labels.get(col, col)
            if col == 'RATE' and pd.notna(value) and value != '':
                if isinstance(value, (int, np.integer)) and value > 0:
                    parts.append(f"{label}: {int(value):,}")
                elif isinstance(value, str) and value.isdigit() and int(value) > 0:
                    parts.append(f"{label}: {int(value):,}")
            elif col == 'MANUYR' and pd.notna(value) and value != '':
                if isinstance(value, (int, np.integer)) and value > 0:
                    parts.append(f"{label}: {int(value)}")
                elif isinstance(value, str) and value.isdigit() and int(value) > 0:
                    parts.append(f"{label}: {int(value)}")
            else:
                parts.append(f"{label}: {value}")

    return ", ".join(parts) if parts else "ข้อมูลไม่เพียงพอ"

def get_classification_details(product_group, gcode):
    product_group_desc = PRODUCT_GROUP_MAPPING.get(product_group, 'ไม่ระบุ')
    gcode_desc = "ไม่ระบุ"
    contno_type = "ไม่ระบุ"
    for type_key, gcode_dict in CONTNO_TYPE_MAPPING.items():
        if gcode in gcode_dict:
            gcode_desc = gcode_dict[gcode]
            contno_type = type_key
            break
            
    return {
        "CONTNO_TYPE": contno_type,
        "GCODE_Description": gcode_desc,
        "Product_Group_Description": f"{product_group}-{product_group_desc}"
    }

def build_response(answer, product_group, gcode):
    classification = get_classification_details(product_group, gcode)
    
    response = f"""
ข้อมูลคำตอบ:
{answer}

รายละเอียดของเพิ่มเติม :
- ประเภทสัญญา (CONTNO_TYPE): {classification['CONTNO_TYPE']}
- รหัสและประเภทย่อย (GCODE): {classification['GCODE_Description']}
- กลุ่มผลิตภัณฑ์: {classification['Product_Group_Description']}
"""
    return response

def create_vector_store():
    car_data = load_data(EXCEL_FILE_PATH)
    texts = [format_row(row) for _, row in car_data.iterrows()]
    documents = [Document(page_content=text, metadata={"id": str(i)}) for i, text in enumerate(texts)]
    
    embed_model = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

    vector_store = FAISS.from_documents(documents, embed_model)
    vector_store.save_local(VECTOR_STORE_PATH)
    return vector_store, embed_model

def load_vector_store():
    embed_model = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

    if os.path.exists(VECTOR_STORE_PATH):
        print("Loading existing vector store...")
        try:
            vector_store = FAISS.load_local(VECTOR_STORE_PATH, embed_model, allow_dangerous_deserialization=True)
            return vector_store, embed_model
        except Exception as e:
            print(f"Error loading vector store: {e}")
            return create_vector_store()
    else:
        return create_vector_store()
    
def build_rag_chain():
    vector_store, _ = load_vector_store()
    retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    
    llm = ChatOpenAI(
        openai_api_key=OPENAI_API_KEY,
        openai_api_base=OPENAI_API_BASE,
        model_name=LLM_MODEL_NAME,
        temperature=0.7,
        max_tokens=4096,
        model_kwargs={"top_p": 0.9}
    )

    template = """
    คุณเป็นผู้ช่วย AI เชี่ยวชาญด้านข้อมูลราคารถ หน้าที่ของคุณคือตอบคำถามเกี่ยวกับราคารถโดยอ้างอิงจากข้อมูลที่ให้มาเท่านั้น

    ข้อมูลราคารถที่เกี่ยวข้อง:
    {context}

    คำถามจากผู้ใช้: {question}

    คำตอบ: (ตอบเป็นภาษาไทย โดยสรุปข้อมูลจาก 'ข้อมูลราคารถที่เกี่ยวข้อง' ที่ตรงกับคำถามมากที่สุด 
    พร้อมระบุ PRODUCT GROUP และ GCODE ที่เกี่ยวข้องด้วย
    อย่าเพิ่มข้อมูลที่ไม่มีอยู่ในข้อมูลที่ให้มา หากไม่พบข้อมูลที่ตรงกัน ให้ตอบว่า "ไม่พบข้อมูลที่เกี่ยวข้อง")
    """
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

def main():
    try:
        rag_chain = build_rag_chain()
        car_data = load_data(EXCEL_FILE_PATH)
        
        while True:
            user_question = input("คำถามของคุณ: ")
            if user_question.lower() in ["exit", "ออก", "ลาก่อน", "พอแล้ว"]:
                break
            if not user_question.strip():
                continue
            
            response = rag_chain.invoke(user_question)
            relevant_data = car_data.iloc[0] 
            product_group = relevant_data.get('PRODUCT GROUP', '')
            gcode = relevant_data.get('GCODE', '')
            
            detailed_response = build_response(response, product_group, gcode)
            
            print("\nคำตอบ:")
            print(detailed_response)
            print("-" * 30)
            
    except Exception as e:
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

C:\Users\kongl\AppData\Local\Temp\ipykernel_13508\1772273683.py:143: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(
c:\Users\kongl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading existing vector store...


C:\Users\kongl\AppData\Local\Temp\ipykernel_13508\1772273683.py:227: UserWarning: Parameters {'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  main()



คำตอบ:

ข้อมูลคำตอบ:
ราคา Toyota Fortuner 3.0 เกียร์ออโต้ ปี 2008 คือ 288,600 บาท โดยเป็นรถประเภท CA ในกลุ่มผลิตภัณฑ์ C (PRODUCT GROUP: C)

รายละเอียดการจำแนกประเภท:
- ประเภทสัญญา (CONTNO_TYPE): C
- รหัสและประเภทย่อย (GCODE): CA_รถเก๋ง (2-5 ประตู)
- กลุ่มผลิตภัณฑ์: C-รถยนต์

------------------------------

คำตอบ:

ข้อมูลคำตอบ:
Honda City 1.0 เกียร์ออโต้ ปี 2020 ราคาประเมินคือ 292,500 บาท โดยอยู่ในกลุ่มผลิตภัณฑ์ C (PRODUCT GROUP: C) และไม่ได้ระบุ GCODE ในข้อมูลที่ให้มา

รายละเอียดการจำแนกประเภท:
- ประเภทสัญญา (CONTNO_TYPE): C
- รหัสและประเภทย่อย (GCODE): CA_รถเก๋ง (2-5 ประตู)
- กลุ่มผลิตภัณฑ์: C-รถยนต์

------------------------------


In [ ]:
import os
import json
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.schema.document import Document

# Configuration
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
JSON_PATH = "Jsonfile\M.JSON"
EXCEL_FILE_PATH = r'Data real\Car rate book.xlsx'
CAR_VECTOR_STORE_PATH = "car_rate_vectorstore"
POLICY_VECTOR_STORE_PATH = "policy_vectorstore"

def create_embeddings_model():
    """Create an embedding model for text embeddings."""
    return HuggingFaceBgeEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True},
        query_instruction="Represent this query for retrieving relevant documents: "
    )

def load_car_data(file_path):
    """Load and preprocess car data from Excel file."""
    if not os.path.exists(file_path):
        print(f"Car rate book file not found: {file_path}")
        return pd.DataFrame()
    
    try:
        df = pd.read_excel(file_path, header=0, dtype=str).fillna('')
        df['MANUYR'] = pd.to_numeric(df['MANUYR'], errors='coerce').astype('Int64')
        df['RATE'] = pd.to_numeric(df['RATE'], errors='coerce').astype('Int64')
        
        try:
            df['FDATEA'] = pd.to_datetime(df['FDATEA'], format='%d-%b-%y', errors='coerce')
            df['LDATEA'] = pd.to_datetime(df['LDATEA'], format='%d-%b-%y', errors='coerce')
        except:
            pass
        
        return df
    except Exception as e:
        print(f"Error loading car data: {e}")
        return pd.DataFrame()

def format_car_row(row):
    """Format a car data row for embedding."""
    columns_labels = {
        'TYPECOD': 'ยี่ห้อ', 'MODELCOD': 'รุ่นหลัก', 'MODELDESC': 'รุ่นย่อย',
        'MANUYR': 'ปีที่ผลิต', 'GEAR': 'เกียร์', 'GCODE': 'ประเภทรถ',
        'PRODUCT GROUP': 'กลุ่มผลิตภัณฑ์', 'RATE': 'ราคาประเมิน'
    }

    parts = []
    for col, label in columns_labels.items():
        value = row.get(col)
        if pd.notna(value) and str(value).strip():
            if col in ['RATE', 'MANUYR']:
                try:
                    num_value = int(value)
                    if num_value > 0:
                        formatted_value = f"{num_value:,}" if col == 'RATE' else str(num_value)
                        parts.append(f"{label}: {formatted_value}")
                except (ValueError, TypeError):
                     parts.append(f"{label}: {value}")
            else:
                parts.append(f"{label}: {value}")
    return ", ".join(parts) if parts else "ข้อมูลไม่เพียงพอ"

def format_value(value):
    """Format various data types for text representation."""
    if isinstance(value, list):
        return "\n".join([f"- {item}" for item in value]) if value else "ไม่มีข้อมูล"
    elif isinstance(value, dict):
        return "\n".join([f"  {k}: {format_value(v)}" for k, v in value.items()]) if value else "ไม่มีข้อมูล"
    else:
        return str(value or "ไม่มีข้อมูล").replace("\\n", "\n")

def parse_json_to_docs(data, parent_key="", docs=None):
    """Parse JSON policy data into document format."""
    if docs is None:
        docs = []

    if isinstance(data, dict):
        current_topic = data.get("หัวข้อ", data.get("หัวข้อย่อย", parent_key.strip('.')))
        content_parts = []
        metadata = {"source": parent_key.strip('.')}

        for key, value in data.items():
            current_key = f"{parent_key}{key}" if parent_key else key
            if isinstance(value, (dict, list)) and key not in ["หัวข้อ", "หัวข้อย่อย"]:
                parse_json_to_docs(value, f"{current_key}.", docs)
            elif key not in ["หัวข้อ", "หัวข้อย่อย"]:
                readable_key = key.replace("_", " ").replace("เป้า ", "เป้าหมาย ")
                content_parts.append(f"{readable_key}: {format_value(value)}")

        if content_parts:
            page_content = f"หัวข้อ: {current_topic}\n" + "\n".join(content_parts)
            docs.append(Document(page_content=page_content.strip(), metadata=metadata))

    elif isinstance(data, list) and parent_key:
        page_content = f"หัวข้อ: {parent_key.strip('.')}\n{format_value(data)}"
        metadata = {"source": parent_key.strip('.')}
        docs.append(Document(page_content=page_content.strip(), metadata=metadata))

    return docs

def build_car_vector_store():
    """Build and save vector store for car rate data."""
    print("Building car rate vector store...")
    car_data = load_car_data(EXCEL_FILE_PATH)
    
    if car_data.empty:
        print("Error: Car data is empty. Vector store not created.")
        return False
        
    texts = [format_car_row(row) for _, row in car_data.iterrows()]
    print(f"Processing {len(texts)} car entries...")
    documents = [Document(page_content=text, metadata={"id": str(i)}) for i, text in enumerate(texts)]
    
    embed_model = create_embeddings_model()
    vector_store = FAISS.from_documents(documents, embed_model)
    
    print(f"Saving car rate vector store to {CAR_VECTOR_STORE_PATH}...")
    vector_store.save_local(CAR_VECTOR_STORE_PATH)
    print("Car rate vector store created successfully.")
    return True

def build_policy_vector_store():
    """Build and save vector store for policy data."""
    print("Building policy vector store...")
    
    try:
        with open(JSON_PATH, "r", encoding="utf-8") as f:
            policy_data = json.load(f)
    except Exception as e:
        print(f"Error loading policy data: {e}")
        return False
    
    documents = parse_json_to_docs(policy_data)
    print(f"Processing {len(documents)} policy documents...")
    
    embed_model = create_embeddings_model()
    vector_store = FAISS.from_documents(documents, embed_model)
    
    print(f"Saving policy vector store to {POLICY_VECTOR_STORE_PATH}...")
    vector_store.save_local(POLICY_VECTOR_STORE_PATH)
    print("Policy vector store created successfully.")
    return True

if __name__ == "__main__":
    print("Starting vector store build process...")
    car_success = build_car_vector_store()
    policy_success = build_policy_vector_store()
    
    if car_success and policy_success:
        print("All vector stores built successfully!")
    else:
        if not car_success:
            print("Failed to build car rate vector store.")
        if not policy_success:
            print("Failed to build policy vector store.")

กำลังส่งคำขอไปยัง LLM เพื่อแปลงข้อมูล...
LLM ตอบกลับเรียบร้อยแล้ว

--- ผลลัพธ์ดิบจาก LLM (Raw String) ---

--- กำลังแปลงสตริงเป็น JSON Object ---

เกิดข้อผิดพลาด: ไม่สามารถแปลงผลลัพธ์จาก LLM เป็น JSON ได้ - Unterminated string starting at: line 59 column 5 (char 1285)
ลองตรวจสอบผลลัพธ์ดิบด้านบน อาจมีรูปแบบที่ไม่ถูกต้อง

--- ผลลัพธ์ดิบที่ได้รับ ---
```json
[
  {
    "section_number": 1,
    "title": "ประเภทรถ",
    "criteria": {
      "เป้าสินเชื่อ ใหม่": "รถมอเตอร์ไซค์ทุกชนิด",
      "เป้า M": "รถมอเตอร์ไซค์ทุกชนิด",
      "เป้า X": null,
      "อำนาจยกเว้น": "ไม่มี"
    },
    "sub_sections": null
  },
  {
    "section_number": 2,
    "title": "ดอกเบี้ย",
    "criteria": {
      "เป้าสินเชื่อ ใหม่": "3.12% ต่อเดือน ไม่มีมากกว่า 3.12%",
      "เป้า M": "3.18% ต่อ 3 เดือน (3.18% /1 เดือน)",
      "เป้า X": "เอื้อย",
      "อำนาจยกเว้น": null
    },
    "sub_sections": null
  },
  {
    "section_number": 3,
    "title": "จำนวนงวด",
    "criteria": {
      "เป้าสินเชื่อ ใหม่": "ไม่เกิน 36